In [4]:
import os
import json
import numpy as np
import pandas as pd
from IPython.display import display, HTML


def display_results_table(data):
  """
  Display the results data as a table sorted by Best Episode Reward.
  """
  # Convert data to a DataFrame and sort it by Best Episode Reward
  df = pd.DataFrame(data)
  if "Landing Success Rate" in df.columns:
    df = df.sort_values(by="Landing Success Rate", ascending=False)

  # Set pandas options to disable scientific notation
  pd.options.display.float_format = '{:,.2f}'.format

  # Display the table in Jupyter Notebook
  display(HTML(df.to_html(index=False, escape=False, border=1)))

  # Reset the formatting to default if needed elsewhere
  pd.reset_option("display.float_format")


def load_results(results_dir="results"):
  """
  Load results from all subdirectories in the specified results directory.
  """
  data = []
  for folder in os.listdir(results_dir):
    folder_path = os.path.join(results_dir, folder)
    if os.path.isdir(folder_path):
      try:
        with open(os.path.join(folder_path, "config.json"), "r") as f:
          config = json.load(f)
        with open(os.path.join(folder_path, "results.json"), "r") as f:
          results = json.load(f)

        data.append({
            "Model": config["model"].upper(),
            "Params": config.get("model_params", ""),
            "Environment": config.get("environment", "N/A"),
            "Episodes": config["episodes"],
            "Average Episode Time": round(results.get("average_time", 0), 2),
            "Reward Strategy": config.get("reward_strategy", "default"),
            "Average Reward": results.get("average_reward", 0),
            "Reward Variance": results.get("variance_in_rewards", 0),
            "Best Episode Training": np.max(results.get("all_rewards", [0])),
            "Best Episode Evaluation": (
                np.max(results.get("eval_rewards", [0]))
                if "eval_rewards" in results else ""
            ),
            "Landing Success Rate": results.get("eval_success_rate", ""),
            "Results Folder": folder_path
        })
      except Exception as e:
        pass

  return data


# Load and display results
results_data = load_results("results")
display_results_table(results_data)

Model,Params,Environment,Episodes,Average Episode Time,Reward Strategy,Average Reward,Reward Variance,Best Episode Training,Best Episode Evaluation,Landing Success Rate,Results Folder
SAC,"{'buffer_size': 1000000, 'batch_size': 256, 'gamma': 0.99, 'tau': 0.005, 'actor_lr': 0.0003, 'critic_lr': 0.0003, 'alpha': 0.2}",LunarLanderContinuous-v3,200,4.78,default,-22.36,"36,111.32",275.83,280.07,0.40,results/SAC-20241126-103559
PPO,"{'gamma': 0.99, 'gae_lambda': 0.95, 'clip_range': 0.2, 'ent_coef': 0.01}",LunarLanderContinuous-v3,50,3.76,default,58.65,"114,191.73",503.05,278.18,0.34,results/PPO-20241125-225439
PPO,"{'gamma': 0.99, 'gae_lambda': 0.95, 'clip_range': 0.2, 'ent_coef': 0.01}",LunarLanderContinuous-v3,200,3.52,default,261.34,"106,853.21",576.13,275.40,0.33,results/PPO-20241126-082047
PPO,"{'gamma': 0.98, 'gae_lambda': 0.92, 'clip_range': 0.15, 'ent_coef': 0.02}",LunarLanderContinuous-v3,200,4.43,default,168.62,"105,818.19",512.38,261.66,0.31,results/PPO-20241126-083638
TOTD,"{'alpha': 0.1, 'gamma': 0.96, 'lambda': 0.85}",LunarLanderContinuous-v3,20,0.01,energy_efficient,-173.78,788.18,-143.35,-141.57,0.22,results/TOTD-20241125-222803
TOTD,"{'alpha': 0.1, 'gamma': 0.97, 'lambda': 0.8}",LunarLanderContinuous-v3,50,0.01,energy_efficient,-179.53,828.39,-141.41,-141.43,0.22,results/TOTD-20241126-012155
TOTD,"{'alpha': 0.1, 'gamma': 0.99, 'lambda': 0.95}",LunarLanderContinuous-v3,100,0.01,proximity,-511.48,"7,660.14",-350.80,-344.28,0.21,results/TOTD-20241126-055451
TOTD,"{'alpha': 0.1, 'gamma': 0.98, 'lambda': 0.9}",LunarLanderContinuous-v3,50,0.01,combined,-436.01,"5,630.03",-324.13,-317.70,0.21,results/TOTD-20241126-023631
TOTD,"{'alpha': 0.1, 'gamma': 0.99, 'lambda': 0.95}",LunarLanderContinuous-v3,100,0.01,combined,-468.02,"7,117.26",-325.03,-316.86,0.21,results/TOTD-20241126-075158
TOTD,"{'alpha': 0.1, 'gamma': 0.97, 'lambda': 0.8}",LunarLanderContinuous-v3,100,0.01,combined,-457.10,"7,387.75",-328.17,-318.22,0.21,results/TOTD-20241126-075204
